In [1]:
from urllib.request import urlopen, Request
from urllib.request import urlopen
import datetime
from time import sleep
from bs4 import BeautifulSoup
import copy
import json
import sys
import csv
import pandas as pd
import numpy as np


In [ ]:
page = urlopen("file:\D:\DP_Data\BitcoinNews\Bitcoin News _ Breaking and Latest News on Bitcoin _ Bitcoinist.html") 
result=[]
soup = BeautifulSoup(page.read(), 'lxml', from_encoding="unicode")
hrefs = soup.find_all('a', class_='featured-image')
for href in hrefs:
    result.append(href['href'])
#print(result)

urlsDf = pd.DataFrame(result, columns =['urls'])


In [ ]:
urlsDf.to_csv("bitcoinistlinks.csv", encoding='utf-8', index=False)

In [85]:
def CheckValidUrl(url):
    if "https://" not in url:
            print("I am skipping wrong html")
            return False
    return True
        
def GetUrlSoup(url):
    headers = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
    req = Request(url=url, headers=headers) 
    page = urlopen(req)
    soup = BeautifulSoup(page.read(), 'lxml', from_encoding="unicode")
    return soup;

def GetAuthorOfPublication(soup):
    author = soup.find('div', class_="meta")
    author = author.find('p')
    author = author.find_all('a')[0]
    author = author.text 
    print(author)
    return author.encode('ascii', 'ignore').decode("utf-8")

def GetDateOfPublish(soup):
    date = soup.find('div', class_="meta")
    date = date.find('p').text.split(" | ")
    date = str(date[1])
    date = datetime.datetime.strptime(date, '%b %d, %Y')
    print(date)
    return date
    
def GetTimeOfPublish(soup):
    date = soup.find('div', class_="meta")
    date = date.find('p').text.split(" | ")
    datetimeText = str(date[1]+ " "+ date[2])
    print(datetimeText)
    date_time = datetime.datetime.strptime(datetimeText, '%b %d, %Y %H:%M')
    print(date_time)
    return date_time

def GetHeader(soup):
    header = soup.find('h1', class_="title").text
    print(header)
    return header.encode('ascii', 'ignore').decode("utf-8")

def GetSubHeaders(soup):
    subHeaders = []
    divContent = soup.find('div', class_="article-content")
    headers = divContent.find_all('h3')
    for header in headers:
        subHeaders.append((header.text).encode('ascii', 'ignore').decode("utf-8"))
    headers = divContent.find_all('h2')
    for header in headers:
        subHeaders.append((header.text).encode('ascii', 'ignore').decode("utf-8"))
    print(subHeaders)
    return subHeaders
    
def GetContent(soup):
    TextContent = ""
    divContent = soup.find('div', class_="article-content")
    textElements = divContent.find_all('p')
    for p in textElements:
        if "What do you think about Bitmain" in p.text or "Images via Shutterstock" in p.text or "The Rundown" in p.text:
            continue
        TextContent = TextContent +" "+ p.text
    #print(TextContent)
    return TextContent.encode('ascii', 'ignore').decode("utf-8")
    
def GetTags(soup):
    tags=[]
    divContent = soup.find('div', class_="article-container") 
    sectionTags = divContent.find('div', class_="tags")
    
    stream = sectionTags.find_all('a')
    for tag in stream:
        tags.append((tag.text).encode('ascii', 'ignore').decode("utf-8"))
    print(tags)
    return tags
    

In [86]:
resultPd = pd.DataFrame(columns =['source','url','author','date','datetime','headertext','subheaders','contenttext','tags'])
with open('bitcoinistlinks.csv') as file:
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        rowPd = pd.DataFrame(columns =['source','url','author','date','datetime','headertext','subheaders','contenttext','tags'])
        crawlurl=row[0]
        if not CheckValidUrl(crawlurl):
            continue;
        soup = GetUrlSoup(crawlurl)
        print(row[0])
        #content Crawl
        Source = "www.bitcoinistlinks.com";
        Url = crawlurl;
        Author = GetAuthorOfPublication(soup);
        Date = GetDateOfPublish(soup);
        DateTime = GetTimeOfPublish(soup);
        HeaderText = GetHeader(soup);
        SubHeadersText = GetSubHeaders(soup);
        ContentText = GetContent(soup);
        Tags = GetTags(soup);
    
    
        rowPd.at[0,'source'] = Source
        rowPd.at[0,'url'] = Url
        rowPd.at[0,'author'] = Author
        rowPd.at[0,'date'] = Date
        rowPd.at[0,'datetime'] = DateTime
        rowPd.at[0,'headertext'] = HeaderText
        rowPd.at[0,'subheaders'] = SubHeadersText
        rowPd.at[0,'contenttext'] = ContentText
        rowPd.at[0,'tags'] = Tags
        #print(Date)
        if "2019-02" in str(Date):
            break
        frames = [resultPd, rowPd]
        resultPd = pd.concat(frames)
        


I am skipping wrong html
https://bitcoinist.com/bitmain-to-increase-hashrate-by-50-with-600000-new-mining-chips/
Julio Gil-Pulgar
2019-08-23 00:00:00
Aug 23, 2019 13:00
2019-08-23 13:00:00
Bitmain to Increase Hashrate By 50% With 600,000 New Mining Chips
['Bitmain to Increase its Capacity By 50% Hash rate', 'Company Hopes to Reach a Valuation of $12 Billion']
['bitcoin', 'bitcoin mining', 'Bitmain', 'TSMC']
https://bitcoinist.com/bitcoin-price-analysis-bullish-divergence-forms-at-10k-is-11k-next/
Smith
2019-08-23 00:00:00
Aug 23, 2019 12:03
2019-08-23 12:03:00
Bitcoin Price Analysis: Bullish Divergence Forms At $10K, is $11K Next?
['Bitcoin 1-Hour Analysis ', '4-Hour Analysis ']
['bitcoin', 'Bitcoin analysis', 'bitcoin TA', 'btcusd', 'xbtusd']
https://bitcoinist.com/us-government-vague-on-bitcoin-role-in-fentanyl-drug-epidemic/
Anja van Oosterhout
2019-08-23 00:00:00
Aug 23, 2019 09:00
2019-08-23 09:00:00
US Government Vague On Bitcoin Role In Fentanyl Drug ‘Epidemic’
['FinCEN Advisory

https://bitcoinist.com/bitcoin-price-analysis-btc-rally-set-for-12000-next-week/
Smith
2019-08-20 00:00:00
Aug 20, 2019 13:46
2019-08-20 13:46:00
Bitcoin Price Analysis: BTC Rally Set For $12,000 Next Week
['Bitcoin 1-Hour Analysis ', '4-Hour Analysis']
['binance', 'Bitcoin analysis', 'BitMEX', 'BTC Analysis', 'btc ta', 'btcusd', 'coinbase', 'huobi', 'Kraken', 'xbtusd']
https://bitcoinist.com/another-case-for-bitcoin-as-billions-vanish-from-savings-accounts-ghana/
Martin Young
2019-08-20 00:00:00
Aug 20, 2019 11:00
2019-08-20 11:00:00
Another Case For Bitcoin as Billions Vanish From Savings Accounts Ghana
['State Cleanup Wipes Out Billions', 'Bitcoin is The Solution']
['bitcoin', 'crackdown', 'Ghana', 'Savings', 'SEC']
https://bitcoinist.com/ethereum-almost-full-says-vitalik-buterin-as-bitcoin-dev-slates-supporters/
Anja van Oosterhout
2019-08-20 00:00:00
Aug 20, 2019 10:00
2019-08-20 10:00:00
Ethereum ‘Almost Full’ Says Vitalik Buterin As Bitcoin Dev Slates Supporters
['Buterin: ETH A

https://bitcoinist.com/coinbase-custody-acquires-xapos-institutional-business/
Emilio Janus
2019-08-16 00:00:00
Aug 16, 2019 06:00
2019-08-16 06:00:00
Coinbase Custody Acquires Xapo’s Institutional Business
['Innovation In Cryptocurrency Custody', 'Coinbases Move Into Crypto-Custody', 'The Final Hurdle To Institutional Adoption?']
['bitcoin', 'coinbase', 'Cryptocurrency', 'Custody', 'Xapo']
https://bitcoinist.com/binance-us-bitcoin-exchange-will-launch-in-couple-of-months-says-cz/
Anja van Oosterhout
2019-08-16 00:00:00
Aug 16, 2019 04:55
2019-08-16 04:55:00
Binance US Bitcoin Exchange Will Launch ‘In Couple Of Months’ Says CZ
['Binance Will Not Serve New York', 'Sympathy For US Regulators']
['binance', 'Binance.US', 'Bitcoin Exchange']
https://bitcoinist.com/brandts-momentum-indicator-highlights-potential-bitcoin-reversal/
Richard Cox
2019-08-15 00:00:00
Aug 15, 2019 18:00
2019-08-15 18:00:00
Brandt’s Momentum Indicator Highlights Potential Bitcoin Reversal
['Bitcoins Line in the Sand

https://bitcoinist.com/goldman-sachs-sets-short-term-bitcoin-price-target-at-13971/
Osato Avan-Nomayo
2019-08-12 00:00:00
Aug 12, 2019 10:00
2019-08-12 10:00:00
Goldman Sachs Sets Short-Term Bitcoin Price Target at $13,971
['30% Bitcoin Price Pullback  Another Buying Opportunity?', 'BTC Bull Case Still Going Strong']
['bitcoin', 'Bitcoin bull market', 'bitcoin price', 'Bitcoin price forecast', 'bitcoin summer weakness', 'Elliot Waves', 'goldman sachs', 'tim draper']
https://bitcoinist.com/new-zealand-is-a-good-place-to-be-paid-in-bitcoin/
Emilio Janus
2019-08-12 00:00:00
Aug 12, 2019 09:00
2019-08-12 09:00:00
New Zealand Is A Good Place To Be Paid In Bitcoin
['Pay Me In Bitcoin', 'Savings In Bitcoin', 'Crypto Salaries Easier Than Ever']
['bitcoin', 'Bitcoin Salaries', 'tax']
https://bitcoinist.com/5-major-bitcoin-exchanges-see-losses-for-first-time-in-2-weeks/
Anja van Oosterhout
2019-08-12 00:00:00
Aug 12, 2019 08:00
2019-08-12 08:00:00
5 Major Bitcoin Exchanges See Losses for First T

https://bitcoinist.com/whale-bets-binance-ceo-10-btc-that-price-will-drop-below-8k/
Anatol Antonovici
2019-08-06 00:00:00
Aug 06, 2019 08:10
2019-08-06 08:10:00
Bitcoin Whale Bets Binance CEO 10 BTC That Price Will Drop Below $8K
['Crazy 10 BTC Bet', 'Bitcoin Continues its Rally for Now']
['binance', 'bitcoin', 'bitcoin price', 'CZ Binance']
https://bitcoinist.com/china-buying-up-bitcoin-narrative-very-dubious/
Emilio Janus
2019-08-06 00:00:00
Aug 06, 2019 08:00
2019-08-06 08:00:00
‘Chinese Buying Up Bitcoin’ Narrative ‘Very Dubious’
['China Unlikely To Be Loading Up On Bitcoin', 'Global Financial Uncertainty Increasing', 'But If It Gives BTC A Boost, Why Not?']
['bitcoin', 'china', 'USD/CNY', 'Yuan']
https://bitcoinist.com/un-north-korea-accumulating-cryptocurrency-for-weapons-programs/
Martin Young
2019-08-06 00:00:00
Aug 06, 2019 07:00
2019-08-06 07:00:00
UN: North Korea Accumulating Cryptocurrency For Weapons Programs
['Cryptocurrency Exchanges Hacked', 'Lazarus On The Attack']
['c

...

https://bitcoinist.com/bitcoin-price-rally-different-silbert/
Emilio Janus
2019-05-18 00:00:00
May 18, 2019 07:00
2019-05-18 07:00:00
3 Reasons The Current Bitcoin Price Rally is Fundamentally Different
['Underlying Bitcoin Infrastructure Improvements', 'Generational Shift In The Way Investors Think About Gold', 'Rising Interest From Institutional Investors']
['Barry Silbert', 'bitcoin investment', 'Grayscale']
https://bitcoinist.com/bitcoin-price-breather-btc-poised-drop-6000/
Martin Young
2019-05-18 00:00:00
May 18, 2019 02:30
2019-05-18 02:30:00
Bitcoin Takes a Breather, But is BTC Poised to Drop to $6,000?
['Trending Down Since Thursday', 'BTC Fash Crashes to Low $6K Range But Bounces Back']
['bitcoin', 'bitcoin price', 'btc', 'correction', 'price']
https://bitcoinist.com/these-13-bitcoin-lightning-network-upgrades-will-solve-its-biggest-hurdles/
Esther Kim
2019-05-17 00:00:00
May 17, 2019 23:00
2019-05-17 23:00:00
These 13 Bitcoin Lightning Network Upgrades Will Solve Its ‘Biggest

https://bitcoinist.com/bitcoin-stablecoins-ecb-presidential-hopeful/
Osato Avan-Nomayo
2019-05-14 00:00:00
May 14, 2019 18:00
2019-05-14 18:00:00
Stablecoins Beat Bitcoin, Says ECB Presidential Hopeful
['We Prefer Fiat-Pegged Crypto', 'Of Course, Central Bankers Prefer Stablecoins to Bitcoin', '$2,000 in Less than Five Days']
['bitcoin', 'ECB', 'Mario Draghi', 'Stablecoins']
https://bitcoinist.com/microsoft-picks-bitcoin-to-create-a-new-identity-system/
Julio Gil-Pulgar
2019-05-14 00:00:00
May 14, 2019 16:00
2019-05-14 16:00:00
Microsoft Picks Bitcoin to Create a New Identity System
['Microsofts New ID Network on the Bitcoin Blockchain', 'Bitcoin is the only unit of value relevant in the operation of the on-chain aspects of the ION network']
['Alex Simons', 'bitcoin', 'blockchain', 'ION', 'microsoft']
https://bitcoinist.com/the-time-has-finally-arrived-so-who-is-bitcoin-creator-satoshi-nakamoto/
Emilio Janus
2019-05-14 00:00:00
May 14, 2019 14:00
2019-05-14 14:00:00
The Time Has Finall

https://bitcoinist.com/hodl-bitcoin-historically-generates-its-yearly-gains-in-10-days-says-tom-lee/
Emilio Janus
2019-05-11 00:00:00
May 11, 2019 11:00
2019-05-11 11:00:00
‘Hodl’ – Bitcoin Historically Generates Its Yearly Gains in 10 Days, Says Tom Lee
['Well That Puts A Bit Of A Downer On Things', 'So, I Havent Missed All The Pump This Year Yet?', 'Golden Cross Foreshadowed The Price Surge']
['10 days', 'annual return', 'bitcoin price', 'btc price', 'Tom Lee']
https://bitcoinist.com/bitcoin-more-mature-asset-now-2017/
Osato Avan-Nomayo
2019-05-11 00:00:00
May 11, 2019 08:00
2019-05-11 08:00:00
4 Examples Bitcoin is Now a More Mature Asset Than in 2017
['Bitcoin Shrugs Off Bitfinex-Tether Fiasco', 'Exchange Hacks No Longer Tank The Market', 'Institutions Are Entering Despite Anti-BTC Sentiment', 'Thanks, Bitcoin Naysayers For The Streisand Effect']
['Binance hack', 'bitcoin', 'Bitcoin Bull Run', 'fud', 'nocoiners', 'Tether']
https://bitcoinist.com/bitcoin-price-parabolic-7000-2019-hi

https://bitcoinist.com/binance-survey-reveals-65-would-donate-to-anti-craig-wright-litigation-fund/
Esther Kim
2019-05-07 00:00:00
May 07, 2019 18:00
2019-05-07 18:00:00
Binance Survey Reveals 65% Would Donate to Anti-Craig Wright Legal Fund
['Zhao Scheme Gets Majority Support', 'Wright Must Disclose Bitcoin Addresses']
['binance', 'Bitcoin SV', 'Changepeng Zhao', 'Craig Wright']
https://bitcoinist.com/can-you-cancel-unconfirmed-bitcoin-transactions-yes-heres-how/


AttributeError: 'NoneType' object has no attribute 'find'

In [83]:
resultPd.head()

,source,url,author,date,datetime,headertext,subheaders,contenttext,tags
0,www.bitcoinistlinks.com,https://bitcoinist.com/bitmain-to-increase-has...,Julio Gil-Pulgar,2019-08-23 00:00:00,2019-08-23 13:00:00,"Bitmain to Increase Hashrate By 50% With 600,0...",[Bitmain to Increase its Capacity By 50% Hash ...,"Julio Gil-Pulgar · @gil_pulgar | Aug 23, 20...","[bitcoin, bitcoin mining, Bitmain, TSMC]"
0,www.bitcoinistlinks.com,https://bitcoinist.com/bitcoin-price-analysis-...,Smith,2019-08-23 00:00:00,2019-08-23 12:03:00,Bitcoin Price Analysis: Bullish Divergence For...,"[Bitcoin 1-Hour Analysis , 4-Hour Analysis ]","Smith | Aug 23, 2019 | 12:03 Bitcoin breaks ...","[bitcoin, Bitcoin analysis, bitcoin TA, btcusd..."
0,www.bitcoinistlinks.com,https://bitcoinist.com/us-government-vague-on-...,Anja van Oosterhout,2019-08-23 00:00:00,2019-08-23 09:00:00,US Government Vague On Bitcoin Role In Fentany...,"[FinCEN Advisory Mentions Cryptocurrencies, Bi...","Anja van Oosterhout | Aug 23, 2019 | 09:00 T...","[bitcoin, drugs, FinCen]"
0,www.bitcoinistlinks.com,https://bitcoinist.com/us-government-vague-on-...,Anja van Oosterhout,2019-08-23 00:00:00,2019-08-23 09:00:00,US Government Vague On Bitcoin Role In Fentany...,"[FinCEN Advisory Mentions Cryptocurrencies, Bi...","Anja van Oosterhout | Aug 23, 2019 | 09:00 T...","[bitcoin, drugs, FinCen]"
0,www.bitcoinistlinks.com,https://bitcoinist.com/eos-is-de-facto-central...,davitbabayan,2019-08-23 00:00:00,2019-08-23 06:00:00,"EOS is De Facto Centralized, Says Wikipedia Co...",[EOS Problem],"davitbabayan | Aug 23, 2019 | 06:00 Wikipedi...","[EOS, Wikipedia]"


In [84]:
resultPd.to_csv("bitcoinist_data.csv", encoding='utf-8', index=False)